In [1]:
from bson import BSON
from pprint import pprint
from facebook_business.adobjects.adaccount import AdAccount

from Core.mongo_adapter import MongoOperator
from Core.mongo_adapter import MongoRepositoryBase
from Core.Web.BusinessOwnerRepository.BusinessOwnerRepository import BusinessOwnerRepository
from Core.Web.FacebookGraphAPI.GraphAPI.GraphAPISdkBase import GraphAPISdkBase
from Core.Web.FacebookGraphAPI.Tools import Tools

class DataRepository(MongoRepositoryBase):
    def get_amount_spent_and_results(self):
        query = {
        }
        projection = {
            "_id": 0,
            "account_id": 1,
            "campaign_id": 1,
            "adset_id": 1,
            "amount_spent": 1,
            "results": 1
        }
        results = self.get(query, projection)
        return results

    def get_campaigns_objective(self, campaign_ids):
        query = {
            MongoOperator.AND.value: [
                {
                    'campaign_id': {
                        MongoOperator.IN.value: campaign_ids
                    }
                },
                {
                    'status': {
                        MongoOperator.IN.value: [1, 2]
                    }
                }
            ]
        }
        projection = {
            "_id": 0,
            "account_id": 1,
            "business_owner_facebook_id": 1,
            "campaign_id": 1,
            "details": 1
        }
        results = self.get(query, projection)
        for index in range(len(results)):
            results[index]['details'] = BSON.decode(results[index]['details'])
            results[index]['objective'] = results[index]['details']['objective']
        return results

    def get_adset_details(self, adset_id):
        query = {
            MongoOperator.AND.value: [
                {
                    'adset_id': {
                        MongoOperator.EQUALS.value: adset_id
                    }
                },
                {
                    'status': 1
                },
            ]
        }
        projection = {
            "_id": 0,
            "details": 1
        }
        response = self.get(query, projection)
        if response:
            response = response[0]
            response = BSON.decode(response['details'])
        else:
            response = {}
        return response

def get_business_owner_id(data, campaign_id):
    bid = next(filter(lambda x: x['campaign_id'] == campaign_id, data))
    if bid:
        return bid['business_owner_facebook_id']

def get_campaign_objective(data, campaign_id):
    obj = next(filter(lambda x: x['campaign_id'] == campaign_id, data))
    if obj:
        return obj['objective']

In [2]:
from Notebooks.FacebookCampaignBudgetEstimation.startup import config, fixtures

repo = DataRepository(config=config.mongo)
repo.database = config.mongo.insights_database_name
repo.collection = 'adset_none_none'
data = repo.get_amount_spent_and_results()

In [3]:
campaign_ids = [entry['campaign_id'] for entry in data]

repo.database = config.mongo.structures_database_name
repo.collection = 'campaign'
campaigns_details = repo.get_campaigns_objective(campaign_ids)

In [18]:
repo.database = config.mongo.structures_database_name
repo.collection = 'adset'

for index in range(len(data)):
    obj = get_campaign_objective(campaigns_details, data[index]['campaign_id'])
    if obj == 'CONVERSIONS' and data[index]['results']:
        data[index]['objective'] = obj
        bid = get_business_owner_id(campaigns_details, data[index]['campaign_id'])
        permanent_token = fixtures.business_owner_repository.get_permanent_token(bid)

        if permanent_token:
            _ = GraphAPISdkBase(config.facebook, permanent_token)
            adset_details = repo.get_adset_details(data[index]['adset_id'])
            targeting_spec = adset_details.get('targeting', None)
            optimization_goal = adset_details.get('optimization_goal', None)

            ad_account_id = "act_" + data[index]['account_id']
            ad_account = AdAccount(fbid=ad_account_id)
            if targeting_spec is not None and optimization_goal is not None:
                try:
                    response = ad_account.get_delivery_estimate(fields=['estimate_mau'],
                                                            params={'targeting_spec': targeting_spec,
                                                                    'optimization_goal': optimization_goal})
                    response = Tools.convert_to_json(response[0])
                    data[index]['audience_size'] = response.get('estimate_mau', None)
                except Exception as e:
                    data[index]['audience_size'] = None
            if not data[index].get('audience_size'):
                data[index]['audience_size'] = None

In [19]:
import typing
from numpy import array
from scipy.interpolate import interp2d

def estimate_budget(objective: typing.AnyStr = None,
                    audience_size: int = None,
                    results: int = None,
                    raw_data: typing.List[typing.Dict] = None) -> int:
    x = [entry['audience_size'] for entry in raw_data if 'objective' in entry.keys() and entry['objective'] == objective and 'audience_size' in entry.keys()]
    x = array(x, dtype=float)

    y = [entry['results'] for entry in raw_data if 'objective' in entry.keys() and entry['objective'] == objective and 'audience_size' in entry.keys()]
    y = array(y, dtype=float)

    z = [entry['amount_spent'] for entry in raw_data if 'objective' in entry.keys() and entry['objective'] == objective and 'audience_size' in entry.keys()]
    z = array(z, dtype=float)

    f = interp2d(x, y, z, kind='cubic')
    return f(audience_size, results)

In [20]:
print("Estimation 1: %s" % estimate_budget(objective='CONVERSIONS', audience_size=10000, results=50, raw_data=data))
# print("Estimation 2: %s" % estimate_budget('PAGE_LIKES', 112411, 200))

Estimation 1: [nan]


In [ ]:
objectives = []
for entry in campaigns_details:
    if entry['objective'] not in objectives:
        objectives.append(entry['objective'])
objectives

In [14]:
import pandas as pd
df = pd.DataFrame(data=data)
df.to_csv("budget-estimation-raw-data-20200723.csv", encoding='utf-8')


NameError: name 'x' is not defined